In [21]:
import urllib
import re
import requests
from urllib.request import Request, urlopen
import numpy as np
import json
import datetime
from dateutil.parser import parse
import pprint
pp = pprint.PrettyPrinter(indent=4)
popUpList = []
information = ["vendor-name", 'type', 'price', 'attribute', 'about', 'menu', 'schedule', 'chef-info', 'stars-count', 'review-count', 'cuisine', 'cuisine-map', 'schedule']
eventVars = ['location', 'address', 'city', 'state', 'latitude', 'longitude', 'times']
timeVars = ['month', 'day', 'startTime', 'endTime']

In [22]:
def price_band(int_):
    if int_ < 25.00:
        return 1
    elif int_ < 35.00:
        return 2
    elif int_ < 60.00:
        return 3
    else:
        return 4

price_band(45)

3

In [23]:
url = 'https://eatfeastly.com/api/v1/meals/upcoming/?non_private=true&market=san-francisco'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
r = urlopen(req).read()
j = json.loads(r)['results']
for popUp in j:
    infoObj = dict.fromkeys(information)
    timeObj = dict.fromkeys(timeVars)
    timeObj['month'] = []
    timeObj['day'] = []
    timeObj['startTime'] = []
    timeObj['endTime'] = []
    eventObj = dict.fromkeys(eventVars)
    eventObj['times'] = timeObj
    infoObj['schedule'] = []
    
    popUpID = str(popUp['pk'])
    popUpUrl = 'https://eatfeastly.com/api/v1/meals/' + popUpID + '/?view=all'
    req = Request(popUpUrl, headers={'User-Agent': 'Mozilla/5.0'})
    r = urlopen(req).read()
    j = json.loads(r)
    
    infoObj['about'] = j['description']
    infoObj['vendor-name'] = j['title']
    infoObj['chef-info'] = j['cook']['about']
    cuisine = []
    for type in j['cuisine_type']:
        cuisine.append(type)
    infoObj['cuisine'] = cuisine
    menuStr = ''
    for dish in j['dishes']:
        menuStr = menuStr + dish['about'] + ' ' + dish['course'] + ' ' + dish['dish_title'] + ' '
    infoObj['menu'] = menuStr
    infoObj['price'] = price_band(float(j['price'][0]['price_feastly']))
    infoObj['review-count'] = j['cook']['review_count']
    eventObj['address'] = j['neighborhood']
    eventObj['city'] = j['location']['market']
    eventObj['latitude'] = j['location']['lat']
    eventObj['longitude'] = j['location']['lng']
    eventObj['state'] = j['location']['state']
    eventObj['times']['day'].append(parse(j['event_data']['start_time']).day)
    eventObj['times']['month'].append(parse(j['event_data']['start_time']).month)
    eventObj['times']['startTime'].append(parse(j['event_data']['start_time']).hour)
    duration = datetime.timedelta(minutes = j['event_data']['duration'])
    endTime = parse(j['event_data']['start_time']) + duration
    eventObj['times']['endTime'].append(endTime.hour)
    
    if len(eventObj) != 0:
        infoObj['schedule'].append(eventObj)
    else:
        infoObj['schedule'].append({})
    infoObj['stars-count'] = j['cook']['review_avg']
    infoObj['type'] = 'popup'
    infoObj['attribute'] = {}
    infoObj['cuisine']
    
    popUpList.append(infoObj)

popUpList

[{'about': 'Experience the flavors of Northern Iran with this 8-course taste of spring! The cuisine of this coastal and mountainous region is one of the most unique in the Middle East. It is characterized by rice, tea, pomegranates, herbs, nuts and citrus.\n\nExperience roasted trout with sour orange molasses, marinated green olives, foraged borage tea, and more.',
  'attribute': {},
  'chef-info': "My name is Hanif Sadr, I'm a chef and co-founder at Komaaj. I started working as a cook at Golestankids (Center for Language Immersion and Cultural Education) in Berkeley and worked for two years. I am also a part time  cooking teacher for Golestankids after school program. I have a bachelor degree in Material Engineering. I was working as an engineer for French Company on weekdays and \r\nFreelance Nature explorer on weekends before I moved to the States. Food was always one of the main topics for me and I've done some research about Northern Iranian cuisine, as my grandparents come from t

In [24]:
%store popUpList

Stored 'popUpList' (list)


In [25]:
popUpJSON = json.dumps(popUpList)
with open('popup-data.txt', 'w') as outfile:
    json.dump(popUpJSON, outfile)

In [26]:

pp.pprint(popUpJSON)

('[{"vendor-name": "Northern Iranian Spring Dinner", "type": "popup", "price": '
 '3, "attribute": {}, "about": "Experience the flavors of Northern Iran with '
 'this 8-course taste of spring! The cuisine of this coastal and mountainous '
 'region is one of the most unique in the Middle East. It is characterized by '
 'rice, tea, pomegranates, herbs, nuts and citrus.\\n\\nExperience roasted '
 'trout with sour orange molasses, marinated green olives, foraged borage tea, '
 'and more.", "menu": "Persian Cucumber, Radish, Daikon, Blood orange and '
 'Carrots  with Cilantro-Mint Dressing, Almonds and Barberries.\\nServed with '
 'green Almonds and green sour Plums. Appetizer Khiaar Daalaar Sumac roasted '
 'Eggplant stuffed with Herbs, Walnut and Pomegranate molasses. served with '
 'Onion sprouts. Small Plate Bademjoon Kabob Green Olives marinated with '
 'pomegranate Molasses, walnut and herbs. Appetizer Zeitun Parvardeh '
 'Saut\\u00e9ed green garlic and garlic chives, cooked with Spin

 '@lauraandsayat", "stars-count": 4.98, "review-count": 3904, "cuisine": '
 '["mediterranean", "middle eastern", "turkish"], "cuisine-map": null}, '
 '{"vendor-name": "ICHIDO Japanese Omakase", "type": "popup", "price": 4, '
 '"attribute": {}, "about": "This tasting menu will feature a bounty of  '
 "seafood, with highlights from the farmer's market, and cuts from the "
 'butcher.\\n\\n\\nPlease keep in mind that courses change based upon the '
 "catch and availability of ingredients and final menu isn't set until the day "
 'of the dinner. The following is a guideline of what you might expect. As a '
 'seafood focused menu, no vegetarian options are available. Please reach out '
 'about any allergies or dietary restrictions.", "menu": "Seared scallop, '
 'ginger, ponzu, stone fruit\\nRockfish ceviche, shiso pickled onion\\nGrilled '
 'octopus, charred onion, kukui nut soy Small Plate Sashimi Trio Cherry '
 'smoked, pickled ginger, wasabi, truffled pea shoots. Small Plate Mt Lassen '
 

In [27]:
pp.pprint (json.loads(popUpJSON)[1])

{   'about': 'Bringing a very unique blend of exotic spices and techniques of '
             'Indian and Mexican fusion in a cosmic Indian Party! All Gluten '
             'Free menu available :)',
    'attribute': {},
    'chef-info': 'Hola! We are Om Sabor and we’re all about cosmic positive '
                 'vibrations and transcendental flavors! My name is Dhanistha '
                 'Rivera and my partner Luis Flores and I are chefs in the '
                 'beautiful San Francisco Bay Area.\r\n'
                 ' \r\n'
                 'Previously popping up as The Vegetarian Blog at music, beer, '
                 'and tequila festivals we created Om Sabor and have both '
                 'joined our passion for cooking unique experiences with plant '
                 'based cuisine. \r\n'
                 '\r\n'
                 "We're excited about our festival feedback and want to take "
                 'this adventure to a new level by sharing our food in a '
         